<a href="https://colab.research.google.com/github/BrooklynZhang/Personal/blob/master/nlp_multi_label_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fast-bert

!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [2]:
!git clone https://github.com/BrooklynZhang/Personal.git

Cloning into 'Personal'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 252 (delta 36), reused 228 (delta 15), pack-reused 0
Receiving objects: 100% (252/252), 2.88 MiB | 5.59 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [0]:
import logging
import torch
import os
import argparse
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy


PATH = 'Personal/NLP_Data/'
OUTPUT = "/content/"

BERT_PRETRAINED_MODEL = "bert-base-uncased"


logger = logging.getLogger()
labels = ["anger", "anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"]
databunch = BertDataBunch(PATH, 
                          PATH,
                          tokenizer=BERT_PRETRAINED_MODEL,
                          train_file='train.csv',
                          label_file='labels.csv',
                          val_file="valid.csv",
                          text_col='text',
                          label_col=labels,
                          batch_size_per_gpu=10,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='bert')

device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(databunch,
                                            pretrained_path=BERT_PRETRAINED_MODEL,
                                            metrics=metrics,
                                            device=device_cuda,
                                            logger=logger,
                                            output_dir=OUTPUT,
                                            finetuned_wgts_path=None,
                                            warmup_steps=200,
                                            multi_gpu=False,
                                            is_fp16=False,
                                            multi_label=True,
                                            logging_steps=10)

learner.fit(epochs=4,
            lr=2e-3,
            schedule_type="warmup_cosine_hard_restarts",
            optimizer_type="lamb")

learner.save_model()